In [105]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler

import h2o
from h2o.automl import H2OAutoML


pd.set_option('display.max_columns', None)
scaler=StandardScaler()
pd.set_option('display.max_columns', None)

In [176]:
train=pd.read_csv('train.csv')

In [177]:
train.dropna(subset=['host_location'],inplace=True)
train.shape

(4155, 74)

In [178]:
train.reset_index(inplace=True)

In [1]:
'''location=[]
for i in range(4155):
    
    r=train.host_location[i].rsplit(' ', 1)[-1]
    print(r)
    train.host_location[i]=r
    location.append(r)'''

"location=[]\nfor i in range(4155):\n    \n    r=train.host_location[i].rsplit(' ', 1)[-1]\n    print(r)\n    train.host_location[i]=r\n    location.append(r)"

In [180]:
l=set(location)
l

{'',
 'Africa',
 'Argentina',
 'Australia',
 'BR',
 'Belgium',
 'Canada',
 'China',
 'Colombia',
 'DE',
 'Denmark',
 'ES',
 'Emirates',
 'FR',
 'Finland',
 'France',
 'GB',
 'GR',
 'Germany',
 'HR',
 'IT',
 'Ireland',
 'Italy',
 'Kingdom',
 'Lithuania',
 'London',
 'NL',
 'NO',
 'Nederland',
 'Netherlands',
 'PL',
 'RS',
 'Romania',
 'Singapore',
 'Spain',
 'States',
 'Sweden',
 'Switzerland',
 'Turkey',
 'US',
 'nb130'}

In [181]:
locatio={'NL':'Netherlands',
         'Netherlands':'Netherlands',
         'US':'US',
         'RS':'Russia',
         'Spain':'Spain',
         'Ireland':'Ireland',
         'States':'US',
         'Switzerland':'Switzerland',
         'Canada':'Canada',
         'Romania':'Romania',
         'Australia':'Australia',
         'Africa':'Unknown',
         'Emirates':'Emirates',
         'Belgium':'Belgium',
         'Lithuania':'Lithuania',
         'China':'China',
         'Singapore':'China',
         'Denmark':'Denmark',
          'GB':'UK',
          'Kingdom':'UK',
         'ES':'Spain',
         'DE':'Germany',
         'BR':'Brazil',
         'London':'UK',
         'Nederland':'Netherlands',
         'FR':'France',
         'IT':'Italy'}

In [182]:
train.host_location=train.host_location.map(locatio)

In [183]:
train.host_location.fillna('Unknown', inplace=True)

In [184]:
train.host_location.value_counts()

Netherlands    4040
Unknown          28
UK               25
US               19
Spain             7
Australia         6
Emirates          4
Germany           4
Switzerland       4
Canada            3
Belgium           3
Romania           2
China             2
Denmark           2
Russia            1
Brazil            1
Ireland           1
Lithuania         1
France            1
Italy             1
Name: host_location, dtype: int64

In [190]:
columnas=['property_type',
 'neighbourhood_cleansed',
 'host_location',
 'availability_30',
 'accommodates',

 'bedrooms',
 'availability_365',
 'room_type', 'reviews_per_month','review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable',]

entreno=train[columnas]

In [191]:
sobras=['property_type_Private room in tiny house',
 'property_type_Entire home/apt',
 'property_type_Boat',
 'property_type_Entire place',
 'host_location_Belgium',
 'property_type_Cave',
 
 'host_location_Italy',
 'host_location_Denmark',
 'property_type_Tower',
 'host_location_Canada',
 'host_location_Ireland',
 'property_type_Shared room in residential home',
 'property_type_Shared room in boat',
 'property_type_Entire cabin',
 'property_type_Casa particular',
 'property_type_Entire chalet',
 'host_location_Russia',
 'host_location_Romania',
 'host_location_France',
 'property_type_Private room in farm stay']

In [192]:
entreno=pd.get_dummies(entreno, columns=['property_type', 'neighbourhood_cleansed','host_location',
        'room_type', 'instant_bookable'], drop_first=True)

entreno.drop(columns=sobras,inplace=True)
X=[c for c in entreno.columns if c!='price']
y='price'

In [193]:
entreno.shape

(4155, 88)

In [194]:
fit=pd.DataFrame(scaler.fit_transform(entreno[X]))

In [195]:
fit

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87
0,-0.823982,0.039404,-0.629656,1.103118,-0.192945,-0.494293,-1.045960,-1.399929,-1.201576,-0.508666,-0.367925,-0.497334,-0.015516,-0.230141,-0.02688,-0.072959,-0.043922,-0.165675,-0.856398,-0.258894,-0.154607,-0.16336,-0.021945,-0.056023,-0.131852,-0.034711,-0.231298,-0.110364,-0.015516,-0.04108,-0.104637,-0.12995,-0.04108,-0.051521,-0.132793,-0.088098,-0.347638,-0.157841,-0.034711,-0.125076,-0.02688,-0.067778,-0.067778,-0.173191,-0.064096,-0.130904,-0.049118,-0.015516,-0.034711,-0.015516,-0.02688,-0.021945,-0.060193,-0.22071,-0.10226,-0.357133,-0.425916,-0.096073,-0.414419,-0.336666,-0.088098,-0.118982,-0.16336,7.326053,-0.147097,-0.203996,-0.083837,-0.217088,-0.247594,-0.128021,-0.162582,-0.261005,-0.266226,-0.015516,-0.021945,-0.031042,-0.031042,-0.015516,0.168717,-0.04108,-0.031042,-0.077803,-0.067778,-0.082369,-0.123076,1.467957,-0.056023,-0.704428
1,-0.510164,0.721594,0.460166,-0.801802,-0.373852,0.586814,0.621605,0.754451,0.058596,0.455793,0.873955,0.799110,-0.015516,-0.230141,-0.02688,-0.072959,-0.043922,-0.165675,1.167682,-0.258894,-0.154607,-0.16336,-0.021945,-0.056023,-0.131852,-0.034711,-0.231298,-0.110364,-0.015516,-0.04108,-0.104637,-0.12995,-0.04108,-0.051521,-0.132793,-0.088098,-0.347638,-0.157841,-0.034711,-0.125076,-0.02688,-0.067778,-0.067778,-0.173191,-0.064096,-0.130904,-0.049118,-0.015516,-0.034711,-0.015516,-0.02688,-0.021945,-0.060193,-0.22071,-0.10226,2.800076,-0.425916,-0.096073,-0.414419,-0.336666,-0.088098,-0.118982,-0.16336,-0.136499,-0.147097,-0.203996,-0.083837,-0.217088,-0.247594,-0.128021,-0.162582,-0.261005,-0.266226,-0.015516,-0.021945,-0.031042,-0.031042,-0.015516,0.168717,-0.04108,-0.031042,-0.077803,-0.067778,-0.082369,-0.123076,-0.681219,-0.056023,-0.704428
2,1.791164,-0.642785,-0.629656,1.242502,-0.072340,-0.010640,-0.022681,0.481360,-1.004674,-0.045726,0.353167,-0.789033,-0.015516,-0.230141,-0.02688,-0.072959,-0.043922,-0.165675,-0.856398,-0.258894,-0.154607,-0.16336,-0.021945,-0.056023,-0.131852,-0.034711,-0.231298,-0.110364,-0.015516,-0.04108,-0.104637,-0.12995,-0.04108,-0.051521,-0.132793,-0.088098,-0.347638,-0.157841,-0.034711,-0.125076,-0.02688,-0.067778,-0.067778,-0.173191,-0.064096,-0.130904,20.359273,-0.015516,-0.034711,-0.015516,-0.02688,-0.021945,-0.060193,-0.22071,-0.10226,-0.357133,-0.425916,-0.096073,-0.414419,-0.336666,-0.088098,-0.118982,-0.16336,-0.136499,-0.147097,-0.203996,-0.083837,-0.217088,-0.247594,-0.128021,-0.162582,-0.261005,3.756207,-0.015516,-0.021945,-0.031042,-0.031042,-0.015516,0.168717,-0.04108,-0.031042,-0.077803,-0.067778,-0.082369,8.125031,-0.681219,-0.056023,1.419591
3,-0.823982,-0.642785,-0.629656,-0.825033,-0.405776,0.586814,0.621605,0.147583,0.491780,0.455793,0.473349,0.183299,-0.015516,-0.230141,-0.02688,-0.072959,-0.043922,-0.165675,1.167682,-0.258894,-0.154607,-0.16336,-0.021945,-0.056023,-0.131852,-0.034711,-0.231298,-0.110364,-0.015516,-0.04108,-0.104637,-0.12995,-0.04108,-0.051521,-0.132793,-0.088098,-0.347638,-0.157841,-0.034711,-0.125076,-0.02688,-0.067778,-0.067778,-0.173191,-0.064096,-0.130904,-0.049118,-0.015516,-0.034711,-0.015516,-0.02688,-0.021945,-0.060193,-0.22071,-0.10226,-0.357133,-0.425916,-0.096073,2.413019,-0.336666,-0.088098,-0.118982,-0.16336,-0.136499,-0.147097,-0.203996,-0.083837,-0.217088,-0.247594,-0.128021,-0.162582,-0.261005,-0.266226,-0.015516,-0.021945,-0.031042,-0.031042,-0.015516,0.168717,-0.04108,-0.031042,-0.077803,-0.067778,-0.082369,-0.123076,-0.681219,-0.056023,-0.704428
4,1.058923,-0.642785,-0.629656,1.877476,0.115661,-0.352042,-0.287976,0.177927,0.294878,0.031431,-1.169138,-0.238045,-0.015516,-0.230141,-0.02688,-0.072959,-0.043922,-0.165675,-0.856398,-0.258894,-0.154607,-0.16336,-0.021945,-0.056023,-0.131852,-0.034711,-0.231298,-0.110364,-0.015516,-0.04108,-0.

In [196]:
fit.columns=X

fit['price']=train.price

In [114]:
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 18.0.1.1+2-6, mixed mode, sharing)
  Starting server from C:\Users\alima\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\alima\AppData\Local\Temp\tmpgt4jlsvk
  JVM stdout: C:\Users\alima\AppData\Local\Temp\tmpgt4jlsvk\h2o_alima_started_from_python.out
  JVM stderr: C:\Users\alima\AppData\Local\Temp\tmpgt4jlsvk\h2o_alima_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,25 days
H2O_cluster_name:,H2O_from_python_alima_qo25af
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.926 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [197]:
h2train=h2o.H2OFrame(fit)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [116]:
automl=H2OAutoML(max_models=50,
                 seed=42,   # random_state
                 max_runtime_secs=300,
                 sort_metric='RMSE')

In [198]:
automl.train(x=X,
             y=y,
             training_frame=h2train)

AutoML progress: |
17:06:53.564: AutoML: XGBoost is not available; skipping it.
17:26:31.863: New models will be added to existing leaderboard AutoML_1_20220508_170653@@price (leaderboard frame=null) with already 27 models.
17:26:31.864: AutoML: XGBoost is not available; skipping it.
17:51:44.884: New models will be added to existing leaderboard AutoML_1_20220508_170653@@price (leaderboard frame=null) with already 54 models.
17:51:44.886: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_15_AutoML_3_20220508_175144


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,53.0,53.0,33642.0,6.0,6.0,6.0,25.0,59.0,45.962265




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 3186.7323477277528
RMSE: 56.45115010101878
MAE: 37.298206670878166
RMSLE: 0.32732505964075226
Mean Residual Deviance: 3186.7323477277528

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 8987.205397483958
RMSE: 94.80087234558529
MAE: 50.28173342859783
RMSLE: NaN
Mean Residual Deviance: 8987.205397483958

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,50.294018,2.624613,47.985165,54.392870,49.571053,51.232070,48.288933
1,mean_residual_deviance,9000.433000,2619.664800,5837.374000,11138.887000,9488.773000,11767.603000,6769.526400
2,mse,9000.433000,2619.664800,5837.374000,11138.887000,9488.773000,11767.603000,6769.526400
3,r2,0.561862,0.285747,0.503462,0.195942,0.412990,0.810631,0.886283
4,residual_deviance,9000.433000,2619.664800,5837.374000,11138.887000,9488.773000,11767.603000,6769.526400
5,rmse,94.021935,14.155746,76.402710,105.540924,97.410340,108.478584,82.277130
6,rmsle,0.410298,0.069166,0.361390,0.459206,NaN,NaN,NaN



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2022-05-08 17:52:57,2.779 sec,0.0,180.856577,79.429099,32709.101353
1,,2022-05-08 17:52:58,2.898 sec,5.0,143.651642,63.474018,20635.794264
2,,2022-05-08 17:52:58,3.057 sec,10.0,109.998556,54.076670,12099.682355
3,,2022-05-08 17:52:58,3.172 sec,15.0,86.614658,49.156302,7502.098902
4,,2022-05-08 17:52:58,3.255 sec,20.0,73.640185,45.515356,5422.876818
5,,2022-05-08 17:52:58,3.334 sec,25.0,69.426118,43.271441,4819.985901
6,,2022-05-08 17:52:58,3.417 sec,30.0,64.588883,41.346391,4171.723766
7,,2022-05-08 17:52:58,3.498 sec,35.0,62.177493,40.163322,3866.040691
8,,2022-05-08 17:52:58,3.575 sec,40.0,60.208455,39.192720,3625.058006
9,,2022-05-08 17:52:58,3.696 sec,45.0,58.652804,38.428641,3440.151363



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,property_type_Room in boutique hotel,1.176570e+08,1.000000,0.209162
1,neighbourhood_cleansed_Centrum-Oost,1.166071e+08,0.991077,0.207295
2,accommodates,4.780642e+07,0.406320,0.084987
3,bedrooms,4.504802e+07,0.382876,0.080083
4,availability_365,4.258536e+07,0.361945,0.075705
5,availability_30,3.903765e+07,0.331792,0.069398
6,room_type_Private room,3.125814e+07,0.265672,0.055568
7,review_scores_value,2.212675e+07,0.188062,0.039335
8,review_scores_location,1.426161e+07,0.121213,0.025353
9,reviews_per_month,8.628468e+06,0.073336,0.015339



See the whole table with table.as_data_frame()


In [210]:
test=pd.read_csv('test.csv')

In [229]:
testeo=test[columnas]

In [230]:
def transfo(train):
    train.dropna(subset=['host_location'],inplace=True)
    train.reset_index(inplace=True)
    for i in range(len(train)):

        r=train.host_location[i].rsplit(' ', 1)[-1]
        print(r)
        train.host_location[i]=r
    train.host_location.fillna('Unknown', inplace=True)
    return train.host_location

In [231]:
testeo.host_location=transfo(testeo)

Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
NL
Netherlands
NL
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
NL
Netherlands
Netherlands
NL
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
NL
Netherlands
Lithuania
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Net


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
NL
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
NL
Netherlands
Netherlands
NL
Netherlands
NL
Netherlands
Netherlands
NL
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
NL
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
NL
NL
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Netherlands
Net

In [232]:
testeo.host_location=testeo.host_location.map(locatio)

In [233]:
testeo.host_location.fillna('Unknown', inplace=True)

In [234]:
testeo.host_location.unique()

array(['Netherlands', 'Lithuania', 'Switzerland', 'China', 'US',
       'Unknown', 'Australia', 'UK', 'Spain', 'Germany', 'Emirates',
       'Brazil'], dtype=object)

In [235]:
testeo=pd.get_dummies(testeo, columns=['property_type', 'neighbourhood_cleansed','host_location',
        'room_type', 'instant_bookable'], drop_first=True)
testeo['property_type_Bus']=0
testeo['property_type_Private room in nature lodge']=0
testeo['property_type_Room in casa particular']=0
testeo['index']=0
testeo['property_type_Private room in bungalow']=0
testeo['property_type_Private room in island']=0



In [236]:
fit_test=pd.DataFrame(scaler.fit_transform(testeo))


In [218]:
sobritas=[]
for i in testeo.columns:
    for j in X:
        if i not in X:
            sobritas.append(i)
sobritas=set(sobritas)

In [221]:
sobritas=list(sobritas)
sobritas

['property_type_Bus',
 'property_type_Private room in nature lodge',
 'property_type_Room in casa particular',
 'index',
 'property_type_Private room in bungalow',
 'property_type_Private room in island']

In [237]:
h2test=h2o.H2OFrame(fit_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [158]:
print('[INFO] Leader board:')

leader_board=automl.leaderboard

leader_board.head()

[INFO] Leader board:


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_grid_1_AutoML_1_20220508_170653_model_1,98.9171,9784.6,52.5492,nan,9784.6
GBM_5_AutoML_1_20220508_170653,99.8166,9963.35,52.2419,nan,9963.35
GBM_10_AutoML_2_20220508_172631,101.042,10209.4,51.6062,nan,10209.4
GBM_grid_2_AutoML_2_20220508_172631_model_1,101.063,10213.7,52.4916,nan,10213.7
GBM_grid_2_AutoML_2_20220508_172631_model_3,101.736,10350.1,54.5579,nan,10350.1
GBM_grid_2_AutoML_2_20220508_172631_model_5,103.96,10807.8,53.2435,nan,10807.8
GBM_grid_1_AutoML_1_20220508_170653_model_5,104.126,10842.1,53.2984,nan,10842.1
GBM_grid_2_AutoML_2_20220508_172631_model_17,105.862,11206.7,54.4913,nan,11206.7
GBM_grid_2_AutoML_2_20220508_172631_model_8,107.966,11656.8,54.6003,nan,11656.8
GBM_grid_1_AutoML_1_20220508_170653_model_8,108.001,11664.1,54.7842,nan,11664.1


In [239]:
y_pred=automl.leader.predict(h2test)
automl.leader.summary()
pred= h2o.as_list(y_pred)

gbm prediction progress: | (failed)


OSError: Job with key $03017f00000132d4ffffffff$_8bb8aea5a7713c02d210480b6955444b failed with an exception: java.lang.IllegalArgumentException: Test/Validation dataset has no columns in common with the training set
stacktrace: 
java.lang.IllegalArgumentException: Test/Validation dataset has no columns in common with the training set
	at hex.Model.adaptTestForTrain(Model.java:1768)
	at hex.Model.adaptTestForTrain(Model.java:1576)
	at hex.Model.adaptTestForTrain(Model.java:1572)
	at hex.Model.adaptFrameForScore(Model.java:1899)
	at hex.Model.score(Model.java:1917)
	at water.api.ModelMetricsHandler$1.compute2(ModelMetricsHandler.java:491)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1668)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java:976)
	at jsr166y.ForkJoinPool.runWorker(ForkJoinPool.java:1479)
	at jsr166y.ForkJoinWorkerThread.run(ForkJoinW